In [2]:
# Ensure the virtual environment is activated before running this cell.
# Install the required dependencies:
!pip install langchain-core langchain-ollama

In [3]:
!pip install langchain_chroma


In [7]:
# install panda
!pip install pandas

In [ ]:
# Install the langgraph module
!pip install langgraph

In [7]:
# Pull the required model for embeddings
from ollama import pull

model_name = "mxbai-embed-large"
pull(model_name)

In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from vector import retriever

model = OllamaLLM(model="qwen2.5")

template = """
You are an exeprt in answering questions about a pizza restaurant

Here are some relevant reviews: {reviews}

Here is the question to answer: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

while True:
    print("\n\n-------------------------------")
    question = input("Ask your question (q to quit): ")
    print("\n\n")
    if question == "q":
        break
    
    reviews = retriever.invoke(question)
    result = chain.invoke({"reviews": reviews, "question": question})
    print(result)

In [11]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
import os
import pandas as pd

df = pd.read_csv("realistic_restaurant_reviews.csv")
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
print(df.head())
print(embeddings)
db_location = "./chrome_langchain_db"
add_documents = True
print(f"Add documents: {add_documents}")
print(f"DB location: {db_location}")

if add_documents:
    print("Adding documents to the vector store...")
    documents = []
    ids = []
    
    for i, row in df.iterrows():
        document = Document(
            page_content=row["Title"] + " " + row["Review"],
            metadata={"rating": row["Rating"], "date": row["Date"]},
            id=str(i)
        )
        ids.append(str(i))
        documents.append(document)


vector_store = Chroma(
    collection_name="restaurant_reviews",
    persist_directory=db_location,
    embedding_function=embeddings
)

if add_documents:
    vector_store.add_documents(documents=documents, ids=ids)
    
retriever = vector_store.as_retriever(
    search_kwargs={"k": 5}
)

question = "What do people think about the pizza?"

reviews = retriever.invoke(question)

print(retriever)
print(reviews)